## Notebook for assignment Week 3

###  Segmenting and Clustering Neighborhoods in Toronto

Install Beautiful Soup with Python 3:

In [1]:
!pip install "ipython-beautifulsoup[bs4]"

distributed 1.21.8 requires msgpack, which is not installed.


Specify the URL and Scrape the wikipedia of list of postal codes of Canada

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
import requests
page = requests.get(url)

In [3]:
# Create a Beautiful Soup object from the html# 
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.text, 'html.parser')

# Print out table rows only, pass the 'tr' argument in soup.find_all().
rows = soup.find_all('tr')

In [4]:
# Below is a for loop that iterates through table rows and prints out the cells of the rows
for row in rows:
    row_td = row.find_all('td')

In [5]:
# remove html
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()

In [6]:
# Build a regular expression that finds all the characters inside the < td > html tags  
# and replace them with an empty string for each table row.
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)

In [7]:
# convert the list into a dataframe 
import pandas as pd
df = pd.DataFrame(list_rows)


In [24]:
# Data manipulation and cleaning
df1 = df[0].str.split(',', expand=True)
df1[0] = df1[0].str.strip('[')
col_labels =[]
for col_name in soup.find_all('th'):
    col_labels.append(col_name.text.rstrip('\n'))
all_header = []
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
all_header.append(cleantext2)
df2 = pd.DataFrame(all_header)
df3 = df2[0].str.split(',', expand=True)
frames = [df3, df1]
df4 = pd.concat(frames)
df5 = df4.rename(columns=df4.iloc[0])
df6 = df5.drop(df5.columns[[3,4]], axis=1)
df7 = df6.drop([0, 1,287, 288, 289, 290,291,292,293,294])
df8 = df7.replace('\n]','', regex=True)
df8.rename(columns={"['Postcode'": 'Postcode'},inplace=True)
df8.rename(columns={" 'Borough'": 'Borough'},inplace=True)
df8.rename(columns={" 'Neighbourhood'": 'Neighbourhood'},inplace=True)
df8.head(5)

,Postcode,Borough,Neighbourhood
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [31]:
# remove rows that have borough == not assigned
df9 = df8[df8['Borough']!=' Not assigned']

In [34]:
#Grouping dataframe by postcode and Borough, we apply join function for neighborhood

singlePostcode = df9.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join)
singlePostcode = singlePostcode.reset_index()
singlePostcode

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
